In [57]:
import numpy as np
import pandas as pd
import math
from typing import Tuple

pd.set_option('display.precision', 12)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

In [58]:
# --- Helper Function to Get Difference Table and Coefficients (MODIFIED) ---
def EvenDifference(points, condition=1):
    """
    Calculates the difference table and extracts coefficients.

    Args:
        points (list of tuples): A list of (x, y) data points.
        condition (int): 1 for Forward (top diagonal), 0 for Backward (bottom diagonal).

    Returns:
        tuple: A tuple containing:
            - pandas.DataFrame: The full difference table.
            - list: The list of extracted coefficients.
    """
    x_values = [p[0] for p in points]
    y_values = [p[1] for p in points]
    n = len(y_values)

    # Internal calculation table
    diff_calc_table = np.full((n, n), np.nan)
    diff_calc_table[:, 0] = y_values
    for j in range(1, n):
        for i in range(n - j):
            diff_calc_table[i, j] = diff_calc_table[i+1, j-1] - diff_calc_table[i, j-1]

    # --- NEW: Format the output DataFrame ---
    data = {
        'x_i': x_values,
        'y_i': y_values
    }
    for j in range(1, n):
        data[f'Order {j}'] = diff_calc_table[:, j]
    
    df = pd.DataFrame(data)

    # Extract coefficients (logic is unchanged)
    if condition == 1:
        coefficients = diff_calc_table[0, :].tolist()
    else:
        coeffs = []
        for j in range(n):
            coeffs.append(diff_calc_table[n-1-j, j])
        coefficients = coeffs

    return df, coefficients

# Newton Interpolation for Even-distributed Node

## Algorithm


a. Finite Difference Table

Given $n+1$ points $(x_i, y_i)$ where $x_i = x_0 + i \cdot h$ for a fixed step $h$.

* **Compute $j$-th order forward difference:**
    * $\Delta^0 y_i = y_i$
    * $\Delta^j y_i = \Delta^{j-1} y_{i+1} - \Delta^{j-1} y_i \quad \text{for } j \ge 1$

*  **Select Interpolation Coefficients ($C_i$):**
    * **Forward (Condition 1):** Select the top diagonal.
        $C_i = \Delta^i y_0 \quad \text{for } i=0, \dots, n$
    * **Backward (Condition 0):** Select the bottom diagonal.
        $C_i = \Delta^i y_{n-i} \quad \text{for } i=0, \dots, n$

*(This step is performed by `EvenDifference(points, condition)`)*

b. Polynomial Construction

The algorithm computes the coefficients $a_k$ for the polynomial $P_n(t) = \sum_{k=0}^n a_k t^k$, where $t$ is a transformed variable.

* **Forward (Condition 1):** $t = \frac{x - x_0}{h}$
    $P_n(t) = \sum_{i=0}^n \frac{\Delta^i y_0}{i!} B_i(t)$
    where $B_i(t) = \prod_{k=0}^{i-1} (t - k) = t(t-1)\dots(t-i+1)$

* **Backward (Condition 0):** $t = \frac{x - x_n}{h}$
    $P_n(t) = \sum_{i=0}^n \frac{\Delta^i y_{n-i}}{i!} B_i(t)$
    where $B_i(t) = \prod_{k=0}^{i-1} (t + k) = t(t+1)\dots(t+i-1)$

**Steps:**

1.  Initialize total polynomial coefficients: $N = [0, \dots, 0]$
2.  Initialize basis polynomial: $B = [1.0]$ (representing $B_0(t) = 1$)
3.  For $i = 0$ to $n$:
    -  Get the $i$-th difference $C_i$ from the table (Step 1.2).
    -  Calculate the main coefficient: $D_i = \frac{C_i}{i!}$\\
    -  If $i > 0$, update the basis polynomial $B$ from its previous state $B_{prev}$:
        * **Forward:** $B(t) = B_{prev}(t) \cdot (t - (i-1))$
        * **Backward:** $B(t) = B_{prev}(t) \cdot (t + (i-1))$
    -  Calculate the $i$-th term polynomial: $N_i = D_i \cdot B(t)$
    -  Add to the total polynomial: $N = N + N_i$ (aligning coefficients by degree)
4.  **Output:** The final coefficient list $N = [a_0, a_1, \dots, a_n]$ for $P_n(t)$.

In [59]:
def NewtonInterpolation(points, condition=1):
    """
    Constructs the Newton interpolation polynomial and returns the steps
    and final coefficients as polished Pandas DataFrames.
    """
    n = len(points) - 1
    if n < 0:
        return pd.DataFrame(), pd.DataFrame()

    diff_table, diff_coeffs = EvenDifference(points, condition=condition)

    # --- NEW: Polished data structures for the output DataFrames ---
    steps_data = []
    N_coeffs_var = np.zeros(n + 1, dtype=float)
    B_coeffs_prev = np.array([1.0])

    for i in range(n + 1):
        D_i = diff_coeffs[i] / math.factorial(i)
        if i == 0:
            B_coeffs = np.array([1.0], dtype=float)
        else:
            k = i - 1
            varB = np.concatenate(([0.0], B_coeffs_prev))
            kB = np.concatenate((k * B_coeffs_prev, [0.0]))
            B_coeffs = varB - kB if condition == 1 else varB + kB
        
        B_coeffs_prev = B_coeffs.copy()
        Ni_coeffs = D_i * B_coeffs
        N_coeffs_var[:len(Ni_coeffs)] += Ni_coeffs

        # Append the polished row of data for the steps DataFrame
        steps_data.append({
            'i': i,
            'Diff Coeff': diff_coeffs[i],
            'D_i': D_i,
            'B_i Coeffs': B_coeffs.tolist(),
            'N_i Coeffs': Ni_coeffs.tolist()
        })

    step_pd = pd.DataFrame(steps_data)
    coeff_pd = pd.DataFrame({
        'Degree': np.arange(n + 1),
        'Coeff': N_coeffs_var
    })

    return step_pd, coeff_pd

## Result

In [60]:
points = [(1.4, 0.9523), (1.5, 0.9661), (1.6, 0.9753), (1.7, 0.9838), (1.8, 0.9891), (1.9, 0.9928), (2.0, 0.9)]

# Set the central node index
x0_index = 0
x0_val = points[x0_index][0] #-1 if use backward (condition = 0)
h = points[1][0] - points[0][0]

In [61]:
df, coeffs = EvenDifference(points, condition = 1)

df.style

,x_i,y_i,Order 1,Order 2,Order 3,Order 4,Order 5,Order 6
0,1.400000,0.952300,0.013800,-0.004600,0.003900,-0.006400,0.010500,-0.111100
1,1.500000,0.966100,0.009200,-0.000700,-0.002500,0.004100,-0.100600,nan
2,1.600000,0.975300,0.008500,-0.003200,0.001600,-0.096500,nan,nan
3,1.700000,0.983800,0.005300,-0.001600,-0.094900,nan,nan,nan
4,1.800000,0.989100,0.003700,-0.096500,nan,nan,nan,nan
5,1.900000,0.992800,-0.092800,nan,nan,nan,nan,nan
6,2.000000,0.900000,nan,nan,nan,nan,nan,nan


In [62]:
step_df, final_coeff_df = NewtonInterpolation(points, condition=1)

step_df.style

,i,Diff Coeff,D_i,B_i Coeffs,N_i Coeffs
0,0,0.952300,0.952300,[1.0],[0.9523]
1,1,0.013800,0.013800,"[0.0, 1.0]","[0.0, 0.013799999999999923]"
2,2,-0.004600,-0.002300,"[0.0, -1.0, 1.0]","[-0.0, 0.0022999999999999687, -0.0022999999999999687]"
3,3,0.003900,0.000650,"[0.0, 2.0, -3.0, 1.0]","[0.0, 0.001300000000000005, -0.0019500000000000073, 0.0006500000000000025]"
4,4,-0.006400,-0.000267,"[0.0, -6.0, 11.0, -6.0, 1.0]","[-0.0, 0.0016000000000000458, -0.0029333333333334175, 0.0016000000000000458, -0.0002666666666666743]"
5,5,0.010500,0.000088,"[0.0, 24.0, -50.0, 35.0, -10.0, 1.0]","[0.0, 0.002100000000000102, -0.004375000000000212, 0.0030625000000001484, -0.0008750000000000424, 8.750000000000424e-05]"
6,6,-0.111100,-0.000154,"[0.0, -120.0, 274.0, -225.0, 85.0, -15.0, 1.0]","[-0.0, 0.018516666666666848, -0.04227972222222264, 0.03471875000000034, -0.013115972222222352, 0.002314583333333356, -0.00015430555555555708]"


In [63]:
final_coeff_df.style

,Degree,Coeff
0,0,0.952300
1,1,0.039617
2,2,-0.053838
3,3,0.040031
4,4,-0.014258
5,5,0.002402
6,6,-0.000154


## Further Test

In [64]:
#Horner Test
def synthetic_division(a, c):
    """
    Perform synthetic division for polynomial p(x) with coefficients a,
    evaluated at x = c.

    Parameters:
        a (list[float]): coefficients of p(x) from highest to lowest degree
        c (float): the value to evaluate p(c)

    Returns:
        df (pd.DataFrame): table with columns [i, a_i, b_i*c, b_i]
        p_c (float): value of p(c)
        q_coeff (list[float]): coefficients of q(x) = (p(x) - p(c)) / (x - c)
    """

    n = len(a) - 1
    b = [0.0] * (n + 1)
    bc_values = [""] * (n + 1)

    b[n] = a[n]
    for i in range(n - 1, -1, -1):
        b[i] = a[i] + c * b[i + 1]
        bc_values[i + 1] = b[i + 1] * c

    # Prepare table (i from n to 0)
    df = pd.DataFrame({
        "i": list(range(n, -1, -1)),
        "a_i": [a[i] for i in range(n, -1, -1)],
        "b_i*c": [bc_values[i] for i in range(n, -1, -1)],
        "b_i = a_i + b_(i+1)*c": [b[i] for i in range(n, -1, -1)]
    })

    p_c = b[0]
    q_coeff = b[1:]
    return df, p_c, q_coeff, b

def all_derivatives(a, c):
    """
    Compute all derivatives p^(i)(c) using repeated Horner division
    and display in transposed table format.
    """
    coeffs = a.copy()
    degree = len(a) - 1
    results = []
    b0_list = []
    derivative_list = []

    # Perform repeated synthetic division
    for i in range(degree + 1):
        df, b0, next_coeff, b_all = synthetic_division(coeffs, c)
        results.append(b_all)
        b0_list.append(b0)
        derivative_list.append(b0 * math.factorial(i))
        coeffs = next_coeff
        if len(coeffs) == 0:
            break

    # Pad b_i lists for equal column length
    max_len = max(len(b) for b in results)
    for b in results:
        b.extend([None] * (max_len - len(b)))

    # Create DataFrame horizontally
    df = pd.DataFrame(results).T
    df.columns = [f"i={i}" for i in range(len(results))]

    # Insert first column for original a coefficients
    a_col = a + [None] * (df.shape[0] - len(a))
    df.insert(0, "a_i", a_col)

    # Add b_0 and p^(i)(c) rows
    df.loc["b_0"] = [None] + b0_list
    df.loc["p^(i)(c)"] = [None] + derivative_list

    # Add a row on top showing the value of c
    df.loc["c"] = [c] + [None] * (df.shape[1] - 1)
    df = df.loc[["c"] + [idx for idx in df.index if idx != "c"]]  # Move row to top

    return df

In [65]:
coeff_list = final_coeff_df['Coeff'].tolist()

x_val = 1.43
t_val = (x_val - x0_val) / h

df2 = all_derivatives(coeff_list, t_val)
df2.style

,a_i,i=0,i=1,i=2,i=3,i=4,i=5,i=6
c,0.300000,nan,nan,nan,nan,nan,nan,nan
0,0.952300,0.960311,0.016677,-0.024879,0.025001,-0.010863,0.002124,-0.000154
1,0.039617,0.026702,-0.033416,0.028455,-0.011514,0.002171,-0.000154,nan
2,-0.053838,-0.043048,0.032109,-0.012179,0.002217,-0.000154,nan,nan
3,0.040031,0.035966,-0.012858,0.002263,-0.000154,nan,nan,nan
4,-0.014258,-0.013551,0.002310,-0.000154,nan,nan,nan,nan
5,0.002402,0.002356,-0.000154,nan,nan,nan,nan,nan
6,-0.000154,-0.000154,nan,nan,nan,nan,nan,nan
b_0,nan,0.960311,0.016677,-0.024879,0.025001,-0.010863,0.002124,-0.000154
p^(i)(c),nan,0.960311,0.016677,-0.049758,0.150004,-0.260708,0.254920,-0.111100
